In [105]:
import pandas as pd
import numpy as np
import tensorflow as tf
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from pickle import dump
import warnings
from keras.optimizers import Adam
from keras import models
from keras.layers import Dropout, Dense, GRU, LSTM, Flatten
warnings.filterwarnings('ignore')

## 1. Przygotowanie danych

In [106]:
dataset = pd.read_csv('EURUSD_2000_2023.csv', parse_dates=True)
dataset.Date = pd.to_datetime(dataset.Date, dayfirst=True)
dataset = dataset.set_index(dataset.Date)
dataset = dataset.drop("Date", axis='columns')
dataset

,Open,High,Low,Close,ZigZag,Return_rate,RSI_7,RSI_14,MA_7,MA_14
Date,,,,,,,,,,
2000-01-18,1.01110,1.01550,1.00720,1.01400,1,1.002769,36.106221,41.847381,1.019714,1.024307
2000-01-19,1.01410,1.01650,1.00850,1.01170,1,0.997732,32.411299,39.100201,1.016786,1.023271
2000-01-20,1.01160,1.01880,1.00730,1.01690,1,1.005140,46.777470,47.492823,1.014700,1.022393
2000-01-21,1.01660,1.01930,1.00500,1.00870,1,0.991936,33.627629,38.485863,1.012514,1.020707
2000-01-23,1.00280,1.00450,1.00260,1.00430,1,0.995638,28.595350,34.684738,1.011386,1.018836
...,...,...,...,...,...,...,...,...,...,...
2022-12-07,1.04670,1.05493,1.04427,1.05114,0,1.004223,64.103095,64.267401,1.050111,1.043287
2022-12-08,1.05115,1.05643,1.04892,1.05551,0,1.004157,69.392195,66.614241,1.052003,1.045037
2022-12-09,1.05549,1.05878,1.05032,1.05325,0,0.997859,63.726914,64.263552,1.052074,1.045859


In [107]:
X_value = pd.DataFrame(dataset.iloc[:, :])
X_value = X_value.drop("ZigZag", axis='columns')
y_value = pd.DataFrame(dataset.ZigZag)

In [108]:
X_value

,Open,High,Low,Close,Return_rate,RSI_7,RSI_14,MA_7,MA_14
Date,,,,,,,,,
2000-01-18,1.01110,1.01550,1.00720,1.01400,1.002769,36.106221,41.847381,1.019714,1.024307
2000-01-19,1.01410,1.01650,1.00850,1.01170,0.997732,32.411299,39.100201,1.016786,1.023271
2000-01-20,1.01160,1.01880,1.00730,1.01690,1.005140,46.777470,47.492823,1.014700,1.022393
2000-01-21,1.01660,1.01930,1.00500,1.00870,0.991936,33.627629,38.485863,1.012514,1.020707
2000-01-23,1.00280,1.00450,1.00260,1.00430,0.995638,28.595350,34.684738,1.011386,1.018836
...,...,...,...,...,...,...,...,...,...
2022-12-07,1.04670,1.05493,1.04427,1.05114,1.004223,64.103095,64.267401,1.050111,1.043287
2022-12-08,1.05115,1.05643,1.04892,1.05551,1.004157,69.392195,66.614241,1.052003,1.045037
2022-12-09,1.05549,1.05878,1.05032,1.05325,0.997859,63.726914,64.263552,1.052074,1.045859


In [109]:
y_value = tf.keras.utils.to_categorical(y_value)

In [110]:
y_value

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [111]:
# Normalized the data
X_scaler = MinMaxScaler(feature_range=(-1, 1))
X_scaler.fit(X_value)
X_scale_dataset = X_scaler.fit_transform(X_value)
dump(X_scaler, open('ClassificationData/X_scaler.pkl', 'wb'))

In [112]:
n_steps_in = 10
n_features = X_value.shape[1]
n_steps_out = 1
output_features = 2
print(n_features)

9


In [113]:
def get_X_y(X_data, y_data):
    X = list()
    y = list()

    length = len(X_data)
    for i in range(0, length-n_steps_in-n_steps_out, 1):
        X_value = X_data[i: i + n_steps_in][:, :]
        # y_value = y_data[i + n_steps_in: i + (n_steps_in + n_steps_out)][:, :]
        y_value = y_data[i + (n_steps_in + 1)]
        if len(X_value) == n_steps_in and len(y_value) == output_features:
            X.append(X_value)
            y.append(y_value)

    return np.array(X), np.array(y)

In [114]:
def predict_index(dataset, X_train, n_steps_in, n_steps_out):

    train_predict_index = dataset.iloc[n_steps_in : X_train.shape[0] + n_steps_in + n_steps_out - 1, :].index
    test_predict_index = dataset.iloc[X_train.shape[0] + n_steps_in:, :].index

    return train_predict_index, test_predict_index

In [115]:
def split_train_test(data):
    train_size = round(len(X) * 0.7)
    data_train = data[0:train_size]
    data_test = data[train_size:]
    return data_train, data_test

In [116]:
X, y = get_X_y(X_scale_dataset, y_value)
X_train, X_test, = split_train_test(X)
y_train, y_test, = split_train_test(y)
train_predict_index, test_predict_index, = predict_index(dataset, X_train, n_steps_in, n_steps_out)
print('X shape: ', X.shape)
print('y shape: ', y.shape)
print('X_train shape: ', X_train.shape)
print('y_train shape: ', y_train.shape)
print('X_test shape: ', X_test.shape)
print('y_test shape: ', y_test.shape)
print('index_train shape:', train_predict_index.shape)
print('index_test shape:', test_predict_index.shape)

X shape:  (7129, 10, 9)
y shape:  (7129, 2)
X_train shape:  (4990, 10, 9)
y_train shape:  (4990, 2)
X_test shape:  (2139, 10, 9)
y_test shape:  (2139, 2)
index_train shape: (4990,)
index_test shape: (2140,)


In [117]:
def get_simulation_data(test_index, output_dim):
    org_dataset = pd.read_csv('EURUSD_DATASET.csv', parse_dates=True)
    org_dataset.Date = pd.to_datetime(org_dataset.Date, dayfirst=True)
    org_dataset = org_dataset.set_index(org_dataset.Date)
    org_dataset = org_dataset.drop("ZigZag", axis='columns')

    sim_indexes = pd.DataFrame()
    sim_indexes['Date'] = test_index
    sim_indexes['Date'] = pd.to_datetime(sim_indexes['Date'], dayfirst=True)
    sim_indexes = sim_indexes.set_index(sim_indexes['Date'])
    sim_indexes.drop("Date", axis='columns')

    sim_dataset = org_dataset.loc[org_dataset.index.isin(sim_indexes.index)]
    cutDays = output_dim - 1
    sim_dataset = sim_dataset.iloc[:-cutDays]

    sim_dataset.to_csv("ClassificationData/Simulation_Dataset.csv", index=False)

In [118]:
X

array([[[-0.52497399, -0.52567427, -0.51896754, ..., -0.21667705,
         -0.51180565, -0.50579834],
        [-0.51716961, -0.52308091, -0.51557815, ..., -0.29164398,
         -0.51958126, -0.50858925],
        [-0.52367326, -0.51711618, -0.51870682, ..., -0.06262035,
         -0.525119  , -0.5109567 ],
        ...,
        [-0.55072841, -0.55160788, -0.54060748, ..., -0.48406236,
         -0.54571488, -0.53538192],
        [-0.55463059, -0.55809129, -0.57215487, ..., -0.70001749,
         -0.55508354, -0.54408184],
        [-0.58688866, -0.59387967, -0.6065702 , ..., -0.83150061,
         -0.57101405, -0.55497599]],

       [[-0.51716961, -0.52308091, -0.51557815, ..., -0.29164398,
         -0.51958126, -0.50858925],
        [-0.52367326, -0.51711618, -0.51870682, ..., -0.06262035,
         -0.525119  , -0.5109567 ],
        [-0.51066597, -0.5158195 , -0.52470343, ..., -0.30840845,
         -0.53092226, -0.51549914],
        ...,
        [-0.55463059, -0.55809129, -0.57215487, ..., -

In [119]:
y

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [120]:
np.save("ClassificationData/X_train.npy", X_train)
np.save("ClassificationData/y_train.npy", y_train)
np.save("ClassificationData/X_test.npy", X_test)
np.save("ClassificationData/y_test.npy", y_test)
np.save('ClassificationData/train_predict_index.npy', train_predict_index)
np.save('ClassificationData/test_predict_index.npy', test_predict_index)
get_simulation_data(test_predict_index, n_steps_out)

## 2. Wytrenowanie klasyfikatora

In [121]:
# Load data
X_train = np.load("ClassificationData/X_train.npy", allow_pickle=True)
y_train = np.load("ClassificationData/y_train.npy", allow_pickle=True)
X_test = np.load("ClassificationData/X_test.npy", allow_pickle=True)
y_test = np.load("ClassificationData/y_test.npy", allow_pickle=True)

In [122]:
# LR = 0.0005
BATCH_SIZE = 128
N_EPOCH = 100

input_dim = X_train.shape[1]     # 7
input_feature_size = X_train.shape[2]  # 5
output_dim = y_train.shape[1]    # 1

In [123]:
print(output_dim)

2


In [124]:
y_train

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [125]:
def basic_GRU(input_dim, output_dim, input_feature_size):
    model = models.Sequential()
    model.add(GRU(units=63, return_sequences=True, activation='relu', input_shape=(input_dim, input_feature_size)))
    model.add(Dropout(0.2))
    model.add(GRU(units=250, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(output_dim, activation='softmax'))

    model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    history = model.fit(X_train, y_train, epochs=N_EPOCH, batch_size=BATCH_SIZE, verbose=2, shuffle=False)

    return model, history

In [126]:
## 61.92% ###

def basic_LSTM(input_dim, output_dim, input_feature_size):
    model = models.Sequential()
    model.add(LSTM(units=63, return_sequences=True, activation='relu', input_shape=(input_dim, input_feature_size)))
    model.add(Dropout(0.2))
    model.add(LSTM(250, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(output_dim, activation='softmax'))

    model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    history = model.fit(X_train, y_train, epochs=N_EPOCH, batch_size=BATCH_SIZE, verbose=2, shuffle=False)

    return model, history

In [127]:
# def basic_LSTM(input_dim, output_dim, input_feature_size):
#     model = models.Sequential()
#     model.add(LSTM(units=50, return_sequences=True, input_shape=(input_dim, input_feature_size)))
#     # model.add(Dropout(0.20))
#     model.add(LSTM(40, return_sequences=False))
#     model.add(Dropout(0.20))
#     model.add(Dense(output_dim, activation='softmax'))
#
#     model.compile(optimizer='adam',
#               loss='categorical_crossentropy',
#               metrics=['accuracy'])
#     history = model.fit(X_train, y_train, epochs=N_EPOCH, batch_size=BATCH_SIZE, verbose=2, shuffle=False)
#
#     return model, history

In [128]:
# model, history = basic_GRU(input_dim, output_dim, input_feature_size)
model, history = basic_LSTM(input_dim, output_dim, input_feature_size)
print(model.summary())

Epoch 1/100
39/39 - 4s - loss: 0.7010 - accuracy: 0.4990 - 4s/epoch - 115ms/step
Epoch 2/100
39/39 - 1s - loss: 0.6916 - accuracy: 0.5313 - 1s/epoch - 33ms/step
Epoch 3/100
39/39 - 1s - loss: 0.6779 - accuracy: 0.5733 - 1s/epoch - 33ms/step
Epoch 4/100
39/39 - 1s - loss: 0.6744 - accuracy: 0.5948 - 1s/epoch - 33ms/step
Epoch 5/100
39/39 - 1s - loss: 0.6644 - accuracy: 0.6174 - 1s/epoch - 31ms/step
Epoch 6/100
39/39 - 1s - loss: 0.6602 - accuracy: 0.6257 - 1s/epoch - 32ms/step
Epoch 7/100
39/39 - 1s - loss: 0.6527 - accuracy: 0.6325 - 1s/epoch - 32ms/step
Epoch 8/100
39/39 - 1s - loss: 0.6484 - accuracy: 0.6349 - 1s/epoch - 33ms/step
Epoch 9/100
39/39 - 1s - loss: 0.6486 - accuracy: 0.6341 - 1s/epoch - 34ms/step
Epoch 10/100
39/39 - 1s - loss: 0.6456 - accuracy: 0.6397 - 1s/epoch - 32ms/step
Epoch 11/100
39/39 - 1s - loss: 0.6463 - accuracy: 0.6423 - 1s/epoch - 31ms/step
Epoch 12/100
39/39 - 1s - loss: 0.6411 - accuracy: 0.6425 - 1s/epoch - 32ms/step
Epoch 13/100
39/39 - 1s - loss: 0.63

In [129]:
score = model.evaluate(X_test, y_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

67/67 [==============================] - 1s 9ms/step - loss: 0.8055 - accuracy: 0.6054
Test loss: 0.805486798286438
Test accuracy: 0.605423092842102


In [130]:
# classificatorName = 'Classificator{:.0f}.h5'.format(score[1]*100)
# model.save('Models/'+classificatorName)
model.save('ClassificationData/Models/Classificator30Days.h5')